In [ ]:
import snntorch

device = "cpu"
class Model(nn.Module):
    def __init__(self, beta: float = 0.95):
        super().__init__()
        self.fc = nn.Linear(n_neurons, n_neurons)
        self.lif = snntorch.Leaky(beta=beta)
        self.mem = self.lif.init_leaky()

    def forward(self, x):
        cur = self.fc(x)
        spk, self.mem = self.lif(cur, self.mem)
        return spk


torch.autograd.set_detect_anomaly(False)

model = Model()
model = torch.compile(model)#, mode="max-autotune")
model = model.to(device)
input_static = torch.randn(n_steps, batch_size, n_neurons).to(device) # n_steps

output = []
for step in range(n_steps):
    output.append(model(input_static[step]))
    
torch.stack(output).sum().backward()

        # for inp in x:
        #                 output.append(spk)
        # return torch.stack(output)


In [35]:
from norse.torch.module import LIFBoxCell, LIFBoxParameters
from norse.torch import SequentialState
import torch
import norse

n_neurons = 100
device = "cuda"
n_steps = 100
batch_size = 64
p = LIFBoxParameters(tau_mem_inv=torch.tensor([100.0], device="cuda"), v_leak=torch.tensor([0.0], device="cuda"), 
                     v_th=torch.tensor([1.0], device="cuda"),
                     v_reset=torch.tensor([0.0], device="cuda"), alpha=torch.tensor([100.0], device="cuda"))
model = SequentialState(
    torch.nn.Linear(n_neurons, n_neurons),
    LIFBoxCell(p),
)
model = model.to(device)
model[1].p

LIFBoxParameters(tau_mem_inv=tensor([100.], device='cuda:0'), v_leak=tensor([0.], device='cuda:0'), v_th=tensor([1.], device='cuda:0'), v_reset=tensor([0.], device='cuda:0'), method='super', alpha=tensor([100.], device='cuda:0'), reset_method=<function reset_value at 0x7f74b654a0e0>)

In [36]:
model = torch.compile(model, mode="reduce-overhead")
input_static = torch.randn(n_steps, batch_size, n_neurons).to(device)

model(input_static)

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 

In [ ]:
import torch 
import torch.nn as nn

import sinabs.layers as sl

device = "cuda"
model = nn.Sequential(
    nn.Linear(10, 10),
    sl.LIF(tau_mem=torch.tensor(10.)),
)

model = torch.compile(model)
model.to(device)

model(torch.randn((1, 20, 10), device=device))

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Assume you have a DataFrame df with columns 'x', 'y1', 'y2'
df = pd.DataFrame({
    'x': ['A', 'B', 'C', 'D'],
    'y1': [10, 15, 7, 10],
    'y2': [5, 2, 7, 8]
})

fig = go.Figure()

fig.add_trace(go.Bar(x=df['x'], y=df['y1'], name='y1'))
fig.add_trace(go.Bar(x=df['x'], y=df['y2'], text=df['y2'], textposition='inside', name='y2'))

# Compute totals for each x
totals = df['y1'] + df['y2']

# Add totals as text using scatter plot
fig.add_trace(go.Scatter(
    x=df['x'], y=totals, 
    mode='text',
    text=totals,
    textposition='top center',
    showlegend=False
))

fig.update_layout(barmode='stack')

fig.show()